In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [3]:
train = pd.read_csv('../data/X_train.csv')

In [4]:
import sys
sys.path.append('../tools/')
from model_manager import model_manager as mm

In [5]:
# from sklearn import preprocessing

# user_encoder = preprocessing.LabelEncoder()
# user_encoder.fit(train.UserId.unique())
# item_encoder = preprocessing.LabelEncoder()
# item_encoder.fit(train.ProductId.unique())
# user_mm = mm('user_encoder', '../models', user_encoder)
# user_mm.save()
# item_mm = mm('item_encoder', '../models', item_encoder)
# item_mm.save()

In [5]:
user_mm = mm('user_encoder', '../models')
item_mm = mm('item_encoder', '../models')

In [6]:
user_mm.load()
item_mm.load()

Done
Done


In [7]:
train['UserId'] = user_mm.model.transform(train['UserId'])
train['ProductId'] = item_mm.model.transform(train['ProductId'])

In [8]:
from surprise import Dataset
from surprise import Reader

In [9]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(train[['UserId', 'ProductId', 'Score']], reader)

In [ ]:
param_grid = {'n_epochs':[5, 10, 15, 20], 'lr_all':[0.002, 0.005, 0.007],
             'reg_all':[0.4, 0.6, 0.8]}

In [ ]:
from surprise import SVDpp
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV

In [ ]:
grid_search = GridSearchCV(SVD, param_grid, measures=['RMSE', 'MAE'], cv = 3, joblib_verbose = 4, n_jobs = 8)

In [ ]:
grid_search.fit(data)

In [ ]:
grid_search.best_params

In [ ]:
grid_search.best_score

In [ ]:
grid_search.best_params

In [ ]:
train_set = data.build_full_trainset()

In [ ]:
svd = SVDpp(verbose=True, n_epochs=20, lr_all = 0.007, reg_all = 0.4)

In [ ]:
svd.fit(train_set)

In [ ]:
svd_mm = mm('svd_model', '../models', svd)
svd_mm.save()

In [10]:
svd_mm = mm('svd_model', '../models')
svd_mm.load()

Done


In [ ]:
svd_mm.model.pu.shape

In [11]:
predictions = svd_mm.model.test([data.df.iloc[i].to_list() for i in range(len(data.df))])

In [12]:
predictions = [int(i.est) for i in predictions]

In [13]:
sum(np.where(data.df.Score.values == predictions, 1, 0))/len(data.df)

0.22072466267343954

In [ ]:
sum(abs(np.array(predictions) - data.df.Score.values))

In [ ]:
train

In [ ]:
original = pd.read_csv('../data/X_train.csv')
test = pd.read_csv('../data/X_test.csv')

In [ ]:
set(test.UserId).difference(set(original.UserId))

In [ ]:
set(test.ProductId).difference(set(original.ProductId))

In [ ]:
unknown = test[(test.UserId.isin({'A39PU8IUVZ6J8T', 'AADQXONRGI10Z'})) | (test.ProductId.isin({'B00006L942', 'B001D5DQIY'}))].reset_index(drop = True)

In [ ]:
known = test[~((test.UserId.isin({'A39PU8IUVZ6J8T', 'AADQXONRGI10Z'})) | (test.ProductId.isin({'B00006L942', 'B001D5DQIY'})))][['Id', 'UserId', 'ProductId', 'Score']].reset_index(drop = True)

In [ ]:
def transformer(df, encoders):
    for col, encoder in encoders.items():
        df[col] = encoder.transform(df[col])
    
    return df

In [ ]:
encoders = {'UserId': user_mm.model, 'ProductId': item_mm.model}

In [ ]:
known = transformer(known, encoders)

In [ ]:
known

In [ ]:
test_data = Dataset.load_from_df(known[['UserId', 'ProductId', 'Score']], reader)

In [ ]:
test_data.df

In [ ]:
predictions = svd.test([test_data.df.iloc[i].to_list() for i in range(len(test_data.df))])

In [ ]:
def approximation(x):
    res = int(x * 10) % 10
    if res >= 5:
        return int(x) + 1
    else:
        return int(x)

In [ ]:
known['Score'] = [approximation(i.est) for i in predictions]

In [ ]:
knn_sub = pd.read_csv('../knn/submission.csv')

In [ ]:
final_d = knn_sub.set_index('Id')['Score'].to_dict()

In [ ]:
final_d.update(known.set_index('Id')['Score'].to_dict())

In [ ]:
knn_sub['Score'] = knn_sub.Id.map(final_d)

In [ ]:
knn_sub.to_csv('./submission.csv', index=False)

In [ ]:
svd.qi.shape

In [ ]:
svd.pu.shape

In [ ]:
len(train.UserId.unique())

In [ ]:
len(train.ProductId.unique())